In [1]:
import os
import sys
sys.path.append(os.pardir)

import numpy as np
import torch as th
import torch.distributed as dist

from guided_diffusion import dist_util, logger
from guided_diffusion.image_datasets import load_data_sde, ImageDataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
    create_dse,
)
from torchvision import utils
from easydict import EasyDict
import PIL.Image as Image
import torchvision
from resizer import Resizer
import lpips
from psnrssim import MetricI2I
from utils import *

to_pil = torchvision.transforms.ToPILImage()
device = 'cuda:5'
batch_size = 1

In [2]:
data = load_data_sde(
        data_dir='../FFHQ_partial/',
        batch_size=batch_size,
        image_size=256,
        class_cond=False,
        deterministic=True
    )

In [3]:
from resizer import Resizer

# Resizer
down_N = 32
shape = (batch_size, 3, 256, 256)
shape_d = (batch_size, 3, int(256 / down_N), int(256 / down_N))
down = Resizer(shape, 1 / down_N).to(device)
up = Resizer(shape_d, down_N).to(device)
die = (down, up)

In [4]:
#load domain-specific feature extractor
dse = create_dse(image_size=256,
                    num_class=2,
                    classifier_use_fp16=True,
                    classifier_width=128,
                    classifier_depth=2,
                    classifier_attention_resolutions='32,16,8',
                    classifier_use_scale_shift_norm=True,
                    classifier_resblock_updown=True,
                    classifier_pool='attention',
                    phase='test')
states = th.load('./dse_face2anime.pt')
dse.load_state_dict(states)
dse.to(device)
dse.convert_to_fp16()
dse.eval()
print('Done')

Done


In [ ]:
sample_fn = diffusion.ddim_sample_loop

model_kwargs = {}
save_idx = 0
for idx, batch in enumerate(data):
    num=250
    if idx <= num:
        continue
    if idx > num+10:
        break
    img, cond = batch
    img = img.to(device)
    noise = th.randn_like(img)
    t = th.tensor([20] * img.shape[0], device=device)
    perturbed = diffusion.q_sample(img, t, noise=noise)
    egsde_kwargs = {"dse":dse, "ls":500, "li":2, "die":die}

    model_kwargs = {}
    sample_trg_scratch = sample_fn(
        model_trg_scratch,
        (args_trg_scratch.batch_size, 3, args_trg_scratch.image_size, args_trg_scratch.image_size),
        clip_denoised=args_trg_scratch.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_scratch = ((sample_trg_scratch + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_trans = sample_fn(
        model_trg_trans,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_trans = ((sample_trg_trans + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_1 = sample_fn(
        model_trg_1,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_1 = ((sample_trg_1 + 1) * 127.5).clamp(0, 255).to(th.uint8)

    # model_kwargs = {}
    # sample_trg_2 = sample_fn(
    #     model_trg_2,
    #     (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
    #     clip_denoised=args_trg_trans.clip_denoised,
    #     model_kwargs=model_kwargs,
    #     noise=perturbed,
    #     ref_img=img,
    #     egsde_kwargs=egsde_kwargs,
    #     t_start=0,
    #     t_end=20
    # )
    # sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    img_pil = ((img + 1) * 127.5).clamp(0, 255).to(th.uint8) 

    print(idx)
    display(concat_four_batch(batch_size, img_pil, sample_trg_scratch, sample_trg_trans, sample_trg_1))
    # display(concat_five_batch(batch_size, img_pil, sample_trg_scratch, sample_trg_trans, sample_trg_1, sample_trg_2))

            

In [ ]:
t_end = 20
sample_fn = diffusion.ddim_sample_loop
model_kwargs = {}
save_idx = 0
num=132
for idx, batch in enumerate(data):
    if idx < num:
        continue
    img, cond = batch
    break
    
for _ in range(10):
    img = img.to(device)
    noise = th.randn_like(img)
    t = th.tensor([t_end] * img.shape[0], device=device)
    perturbed = diffusion.q_sample(img, t, noise=noise)
    egsde_kwargs = {"dse":dse, "ls":500, "li":2, "die":die}

    model_kwargs = {}
    sample_trg_scratch = sample_fn(
        model_trg_scratch,
        (args_trg_scratch.batch_size, 3, args_trg_scratch.image_size, args_trg_scratch.image_size),
        clip_denoised=args_trg_scratch.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_scratch = ((sample_trg_scratch + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_trans = sample_fn(
        model_trg_trans,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_trans = ((sample_trg_trans + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_2 = sample_fn(
        model_trg_2,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    img_pil = ((img + 1) * 127.5).clamp(0, 255).to(th.uint8) 

    print(idx)
    display(concat_four_batch(batch_size, img_pil, sample_trg_scratch, sample_trg_trans, sample_trg_2))


In [ ]:
sample_fn = diffusion.ddim_sample_loop

model_kwargs = {}
save_idx = 0
for idx, batch in enumerate(data):
    num=80
    if idx <= num:
        continue
    if idx > num+10:
        break
    img, cond = batch
    img = img.to(device)
    noise = th.randn_like(img)
    t = th.tensor([20] * img.shape[0], device=device)
    perturbed = diffusion.q_sample(img, t, noise=noise)

    # model_kwargs = {}
    # sample_trg_scratch = sample_fn(
    #     model_trg_scratch,
    #     (args_trg_scratch.batch_size, 3, args_trg_scratch.image_size, args_trg_scratch.image_size),
    #     clip_denoised=args_trg_scratch.clip_denoised,
    #     model_kwargs=model_kwargs,
    #     noise=perturbed,
    #     ref_img=img,
    #     egsde_kwargs=egsde_kwargs,
    #     t_start=0,
    #     t_end=20
    # )
    # sample_trg_scratch = ((sample_trg_scratch + 1) * 127.5).clamp(0, 255).to(th.uint8)

    # model_kwargs = {}
    # sample_trg_trans = sample_fn(
    #     model_trg_trans,
    #     (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
    #     clip_denoised=args_trg_trans.clip_denoised,
    #     model_kwargs=model_kwargs,
    #     noise=perturbed,
    #     ref_img=img,
    #     egsde_kwargs=egsde_kwargs,
    #     t_start=0,
    #     t_end=20
    # )
    # sample_trg_trans = ((sample_trg_trans + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_1 = sample_fn(
        model_trg_1,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=20
    )
    sample_trg_1 = ((sample_trg_1 + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_2 = sample_fn(
        model_trg_2,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=20
    )
    sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    img_pil = ((img + 1) * 127.5).clamp(0, 255).to(th.uint8) 

    print(idx)
    display(concat_three_batch(batch_size, img_pil, sample_trg_1, sample_trg_2))
    # display(concat_five_batch(batch_size, img_pil, sample_trg_scratch, sample_trg_trans, sample_trg_1, sample_trg_2))

            

In [ ]:
t_end = 20
sample_fn = diffusion.ddim_sample_loop
model_kwargs = {}
save_idx = 0
num=87
for idx, batch in enumerate(data):
    if idx < num:
        continue
    img, cond = batch
    break
    
row = 1
img = img[row].unsqueeze(0).repeat(batch_size, 1,1,1)

for _ in range(10):
    img = img.to(device)
    noise = th.randn_like(img)
    t = th.tensor([20] * img.shape[0], device=device)
    perturbed = diffusion.q_sample(img, t, noise=noise)

    # model_kwargs = {}
    # sample_trg_scratch = sample_fn(
    #     model_trg_scratch,
    #     (args_trg_scratch.batch_size, 3, args_trg_scratch.image_size, args_trg_scratch.image_size),
    #     clip_denoised=args_trg_scratch.clip_denoised,
    #     model_kwargs=model_kwargs,
    #     noise=perturbed,
    #     ref_img=img,
    #     egsde_kwargs=egsde_kwargs,
    #     t_start=0,
    #     t_end=20
    # )
    # sample_trg_scratch = ((sample_trg_scratch + 1) * 127.5).clamp(0, 255).to(th.uint8)

    # model_kwargs = {}
    # sample_trg_trans = sample_fn(
    #     model_trg_trans,
    #     (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
    #     clip_denoised=args_trg_trans.clip_denoised,
    #     model_kwargs=model_kwargs,
    #     noise=perturbed,
    #     ref_img=img,
    #     egsde_kwargs=egsde_kwargs,
    #     t_start=0,
    #     t_end=20
    # )
    # sample_trg_trans = ((sample_trg_trans + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_1 = sample_fn(
        model_trg_1,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_1 = ((sample_trg_1 + 1) * 127.5).clamp(0, 255).to(th.uint8)

    model_kwargs = {}
    sample_trg_2 = sample_fn(
        model_trg_2,
        (args_trg_trans.batch_size, 3, args_trg_trans.image_size, args_trg_trans.image_size),
        clip_denoised=args_trg_trans.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        ref_img=img,
        egsde_kwargs=egsde_kwargs,
        t_start=0,
        t_end=20
    )
    sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    img_pil = ((img + 1) * 127.5).clamp(0, 255).to(th.uint8) 

    print(idx)
    display(concat_three_batch(batch_size, img_pil, sample_trg_1, sample_trg_2))
